# "Fast and Efficient Inequality Joins in Pandas"
> "Compute inequality joins better than a cross join or a naive nested loop."

- toc: true
- branch: master
- badges: true
- hide_binder_badge: True
- hide_colab_badge: True
- comments: true
- author: Samuel Oranyeli
- categories: [python, Pandas, non-equi join, inequality join, conditional join, equi-join]
- hide: false
- search_exclude: true
- metadata_key1: "python Pandas"
- metadata_key2: "joins"

## **Introduction**

Pandas supports *equi-join*, where the keys involved in the join are considered to be *equal*. This is implemented via the [merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) and [join](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html#pandas.DataFrame.join) functions. There are scenarios however, where the keys involved might not be equal; some other logical condition is involved in the join. This is known as a *non-equi join* or an *inequality join*. 

Let's look at some examples, culled from [here](https://github.com/pandas-dev/pandas/issues/34543#issue-629846136), that illustrate equi and non-equi joins:

In [1]:
import pandas as pd

df1 = pd.DataFrame({'left': [1,2,3], 'col_a': ["A", "B", "C"]})
df2 = pd.DataFrame({'right': [0, 2, 3], 'col_b': ["Z", "X", "Y"]})

In [2]:
df1

,left,col_a
0,1,A
1,2,B
2,3,C


In [3]:
df2

,right,col_b
0,0,Z
1,2,X
2,3,Y


Join based on the conditional `df1.left == df2.right` - this is an equi join, because the join operator is an *equality* operator, where df1.left is *equal to* df2.right:

| left | col_a | right | col_b |
|----- |:------:|:-----:|-----:|
|   2  |   B   |   2   |  X    |
|   3  |   C   |   3   |  Y    |


Join based on the conditional `df1.left != df2.right` - this is an example of a non-equi join; it should return rows where df1.left *is not equal to* df2.right:

| left | col_a | right | col_b |
|----- |:------:|:-----:|-----:|
|   1  |   A   |   2   |  X    |
|   1  |   A   |   3   |  Y    |
|   2  |   B   |   3   |  Y    |
|   1  |   A   |   0   |  Z    |
|   2  |   B   |   0   |  Z    |
|   3  |   C   |   0   |  Z    |
|   3  |   C   |   2   |  X    |

Join based on the conditional `df1.left > df2.right` - this is another example of a non-equi join; it should return rows where df1.left *is greater than* df2.right:

| left | col_a | right | col_b |
|----- |:------:|:-----:|-----:|
|   1  |   A   |   0   |  Z    |
|   2  |   B   |   0   |  Z    |
|   3  |   C   |   0   |  Z    |
|   3  |   C   |   2   |  X    |

non-equi joins are supported in SQL, and offer elegant solutions for specific uses cases. A good example is provided [here](https://github.com/pandas-dev/pandas/issues/34543#issue-629846136) - 

Consider a dataframe `df_process` that contains `process_id`, `process_start_date` and `process_end_date` columns for some business process, and a second dataframe `df_events` that contains an `event_id` and an `event_date` column for particular events. If you want to find all events that occur during some business process, you can easily obtain these by applying the conditional join statement:


``df_event.event_date >= df_process.process_start_date & df_event.event_date <= df_process.process_end_date``


The above scenario is an example of a range join, where the join is *between a range of values*. Currently in Pandas, to solve non-equi joins, a cartesian join is employed, where every row from the left dataframe is joined to every row from the right dataframe, before it is filtered with the non-equi joins: 

```py
(df_event
.merge(df_process, how = 'cross')
.query('process_start_date <= event_date <= process_end_date')
)
```

This is not an efficient way to deal with inequality joins, in terms of memory and speed, especially for large data. This is where the [conditional_join](https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.conditional_join.conditional_join) function from [pyjanitor](https://pyjanitor-devs.github.io/pyjanitor/) comes into play. It provides an efficient way to deal with non-equi joins, and does so in a more memory efficient way than cartesian joins, while still being performant. Under the hood it uses binary search, and in some special cases, such as range joins, it uses special algorithms to improve performance.

We will be using the [dev](https://github.com/pyjanitor-devs/pyjanitor.git) version in this article.



## **Strictly Non-equi Joins**

Let's load the [pyjanitor](https://pyjanitor-devs.github.io/pyjanitor/) library:

In [4]:
# install the dev version
# to access the latest developments
# pip install git+https://github.com/pyjanitor-devs/pyjanitor.git
import janitor as jn
import sys

In [5]:
print(" pandas version :", pd.__version__,"\n", 
      "janitor version :", jn.__version__, "\n",
      "python version :", sys.version)

 pandas version : 1.4.4 
 janitor version : 0.23.1 
 python version : 3.9.12 | packaged by conda-forge | (main, Mar 24 2022, 23:23:20) 
[Clang 12.0.1 ]


Let's walk through some of the parameters in [conditional_join](https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.conditional_join.conditional_join):

        - df : Left DataFrame.
        - right: Named Series or DataFrame to join to `df`.
        - conditions : Variable argument of tuple(s) of the form `(left_on, right_on, op)`- 
                       `left_on` is the column from `df`, `right_on` is the column from `right`, 
                       while `op` is the join operator. For multiple conditions, the and(&) operator
                       is used to combine the results of the individual conditions.
        - how: indicates the type of join to be performed. Can be one of `left`, `right`, or `inner`.
        - use_numba: Whether to use `numba`, for possible performance increase. Applicable only to strictly non-equi joins.
        - df_columns: Select columns from `df`.
        - right_columns: Select columns from `right`.


Let's apply the [conditional_join](https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.conditional_join.conditional_join) function to solve the non-equi joins earlier:

In [6]:
df1.conditional_join(df2, ('left', 'right', '!='))

,left,col_a,right,col_b
0,1,A,2,X
1,1,A,3,Y
2,2,B,3,Y
3,1,A,0,Z
4,2,B,0,Z
5,3,C,0,Z
6,3,C,2,X


In [7]:
df1.conditional_join(df2, ('left', 'right', '>'))

,left,col_a,right,col_b
0,1,A,0,Z
1,2,B,0,Z
2,3,C,0,Z
3,3,C,2,X


For multiple non-equi conditions, the results of the individual conditions are combined with the and(&) operator. Let's look at a range join, to see this in action - the example is adapted from [here](http://da.qcri.org/ntang/pubs/vldbj2016.pdf):

In [8]:
east = dict(id = range(100,103), 
            dur = [140, 100, 90], 
            rev = [9, 12, 5], 
            cores = [2, 8, 4])

west = dict(t_id = [404,498, 676, 742], 
            time = [100, 140, 80, 90], 
            cost = [6, 11, 10, 5], 
            cores = [4, 2, 1, 4])

east = pd.DataFrame(east)
west = pd.DataFrame(west)

In [9]:
east

,id,dur,rev,cores
0,100,140,9,2
1,101,100,12,8
2,102,90,5,4


In [10]:
west

,t_id,time,cost,cores
0,404,100,6,4
1,498,140,11,2
2,676,80,10,1
3,742,90,5,4


Task : Look for any customer from the West Coast who rented a virtual machine for more hours than any customer from the East Coast, but who paid less. Basically, get rows where `east.dur < west.time` and `east.rev > west.cost`. 

This is a range join, and is solved easily and efficiently with [conditional_join](https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.conditional_join.conditional_join):

In [11]:
(east
.conditional_join(
    west, 
    ('dur', 'time', '<'), 
    ('rev', 'cost', '>'), 
    df_columns = ['id', 'dur', 'rev'],
    right_columns = ['t_id', 'time', 'cost']
    )
)

,id,dur,rev,t_id,time,cost
0,101,100,12,498,140,11


For range joins, and when `use_numba = False`, [conditional_join](https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.conditional_join.conditional_join) uses an algorithm based on this [vertica blog post](https://www.vertica.com/blog/what-is-a-range-join-and-why-is-it-so-fastba-p223413/). 

You might get more performance with `use_numba = True`. The implementation is based on the algorithm in this [publication](https://www.scitepress.org/papers/2018/68268/68268.pdf)

## **Equi and Non-equi Joins**

[conditional_join](https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.conditional_join.conditional_join) also supports a combination of equi and non-equi join conditions - under the hood it uses Pandas' internal merge function to get the matching pairs, before pruning the non-equi joins to get the final dataframe. Depending on the size of the dataframes, this might offer more performance than the classic merge and filter approach:

In [12]:
df1 = pd.DataFrame({'id': [1,1,1,2,2,3], 
                    'value_1': [2,5,7,1,3,4]})
                    
df2 = pd.DataFrame({'id': [1,1,1,1,2,2,2,3], 
                    'value_2A': [0,3,7,12,0,2,3,1], 
                    'value_2B': [1,5,9,15,1,4,6,3]})

In [13]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,3
5,3,4


In [14]:
df2

,id,value_2A,value_2B
0,1,0,1
1,1,3,5
2,1,7,9
3,1,12,15
4,2,0,1
5,2,2,4
6,2,3,6
7,3,1,3


Classic Pandas' merge and filter approach:

In [15]:
(df1
.merge(df2, on = 'id')
.query('value_2A <= value_1 <= value_2B')
)

,id,value_1,value_2A,value_2B
5,1,5,3,5
10,1,7,7,9
12,2,1,0,1
16,2,3,2,4
17,2,3,3,6


With [conditional_join](https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.conditional_join.conditional_join):

In [16]:
(df1
.conditional_join(
    df2,
    ('id', 'id', '=='),
    ('value_1', 'value_2A', '>='),
    ('value_1', 'value_2B', '<=')
))

left         right                  
    id value_1    id value_2A value_2B
0    1       5     1        3        5
1    1       7     1        7        9
2    2       1     2        0        1
3    2       3     2        2        4
4    2       3     2        3        6

Note the above returned a MultiIndex column; if the columns from the left and right dataframes have something in common a MultiIndex column is returned. Of course, you can select/rename columns with `df_columns` and `right_columns` to get a single index column, if that is preferred.

Note that there is no numba implementation when an equi join condition is present.

## **Non-equi joins with OR**

For multiple non-equi joins, an and(&) operator is applied, to combine the results of the individual conditions. There are scenarios however, where the join might have an *OR* condition. In this case, the joins are executed individually, and the resulting dataframes can then be concatenated into one. Let's look at an example from [Nelson Tang's blog post](https://www.nelsontang.com/blog/2021-11-25-pandas-conditional-merging):

In [17]:
sales_volume_table = pd.DataFrame.from_dict([
    {'date':'2021-11-15', 'quantity':1, 'brand':'Outdoor'},
    {'date':'2021-11-20', 'quantity':2, 'brand':'Leisure'},
    {'date':'2021-11-25', 'quantity':3, 'brand':'Athletic'},
    {'date':'2021-11-26', 'quantity':2, 'brand':'Outdoor'},
])

promos_table = pd.DataFrame.from_dict([
    {'start_date':'2021-11-01', 'end_date':'2021-11-25',
    'brand':'ANY', 'rebate_per_unit':3},
    {'start_date':'2021-11-25', 'end_date':'2021-11-26',
    'brand':'Outdoor', 'rebate_per_unit':5},
])

sales_volume_table['date'] = pd.to_datetime(sales_volume_table['date'])
promos_table['start_date'] = pd.to_datetime(promos_table['start_date'])
promos_table['end_date'] = pd.to_datetime(promos_table['end_date'])


In [18]:
sales_volume_table

,date,quantity,brand
0,2021-11-15,1,Outdoor
1,2021-11-20,2,Leisure
2,2021-11-25,3,Athletic
3,2021-11-26,2,Outdoor


In [19]:
promos_table

,start_date,end_date,brand,rebate_per_unit
0,2021-11-01,2021-11-25,ANY,3
1,2021-11-25,2021-11-26,Outdoor,5


Problem statement, culled from the blog post: 
<blockquote>
Here's a simplified version of the issue I was facing:

1. The date in the left table was between two dates (a start and end date) in the second table

2. ...AND the values in two other columns matched each other, OR the column on the right table was equal to 'ANY' (aka a 'wildcard' value)
</blockquote>

Translating this into SQL is [easy](https://www.nelsontang.com/blog/2021-11-25-pandas-conditional-merging): 
```SQL
SELECT *
FROM sales_volume_table, promos_table
WHERE (sales_volume_table.brand=promos_table.brand or promos_table.brand='ANY')
AND (start_date <= date AND date <= end_date)
```

Replicating this in Pandas is more involved but doable:

In [20]:
top = (sales_volume_table
       .conditional_join(
            promos_table,
            ('brand', 'brand', '=='),
            ('date', 'start_date', '>='),
            ('date', 'end_date', '<=')
          )
    )

bottom = (sales_volume_table
          .conditional_join(
               promos_table.query('brand == "ANY"'),
               ('date', 'start_date', '>='),
               ('date', 'end_date', '<=')
               )
     )

pd.concat([top, bottom])

left                         right                                    
        date quantity     brand start_date   end_date    brand rebate_per_unit
0 2021-11-26        2   Outdoor 2021-11-25 2021-11-26  Outdoor               5
0 2021-11-15        1   Outdoor 2021-11-01 2021-11-25      ANY               3
1 2021-11-20        2   Leisure 2021-11-01 2021-11-25      ANY               3
2 2021-11-25        3  Athletic 2021-11-01 2021-11-25      ANY               3

## **Performance**

Why bother with [conditional_join](https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.conditional_join.conditional_join)? memory and speed efficiency. Let's see that in action, compared to a cartesian join:

In [21]:
%load_ext memory_profiler
from string import ascii_lowercase
import numpy as np
np.random.seed(1)
n = 2_000; k = 10_000

idx1 = np.random.randint(0, high = 100, size = n)
idx2 = np.random.randint(0, high = 100, size = n)

d1 = dict(x = np.random.choice(list(ascii_lowercase[:5]), size=n),
          start = np.minimum(idx1, idx2),
          end = np.maximum(idx1, idx2))

d2 = dict(x = np.random.choice(list(ascii_lowercase[:15]), size=k),
          pos1 = np.random.randint(low=60, high = 151, size=k))

d1 = pd.DataFrame(d1)
d2 = pd.DataFrame(d2)

In [22]:
d1.head()

,x,start,end
0,c,37,61
1,d,12,16
2,b,72,79
3,c,9,74
4,d,75,76


In [ ]:
d2.head()

,x,pos1
0,f,126
1,m,121
2,o,128
3,a,96
4,n,61


In [23]:
len(d1), len(d2)

(2000, 10000)

In [24]:
A = (d1
    .merge(d2, how = 'cross')
    .query("start <= pos1 <= end")
    .filter(['start', 'end', 'pos1'])
    )
A.shape

(2740898, 3)

In [25]:
A.head()

,start,end,pos1
4,37,61,61
53,37,61,60
74,37,61,60
104,37,61,60
105,37,61,60


In [26]:
B = (d1
    .conditional_join(
        d2, 
        ('start', 'pos1', '<='), 
        ('end', 'pos1', '>='), 
        df_columns=['start', 'end'],
        right_columns='pos1',
        use_numba=False)
    )
B.shape

(2740898, 3)

In [27]:
B.head()

,start,end,pos1
0,37,61,60
1,37,61,60
2,37,61,60
3,37,61,60
4,37,61,60


In [28]:
C = (d1
    .conditional_join(
        d2, 
        ('start', 'pos1', '<='), 
        ('end', 'pos1', '>='), 
        df_columns=['start', 'end'],
        right_columns='pos1',
        use_numba=True)
    )
C.shape

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


(2740898, 3)

Check that the outputs match:

In [29]:
cols = ['start', 'end', 'pos1']
A = A.sort_values(cols, ignore_index=True)
B = B.sort_values(cols, ignore_index=True)
C = C.sort_values(cols, ignore_index=True)
A.equals(B), A.equals(C), B.equals(C)

(True, True, True)

Let's compare the speed:

In [30]:
%timeit d1.merge(d2, how = 'cross').query("start <= pos1 <= end")

3.39 s ± 84.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%timeit d1.conditional_join(d2, ('start', 'pos1', '<='), ('end', 'pos1', '>='), use_numba=False)

114 ms ± 2.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [32]:
%timeit d1.conditional_join(d2, ('start', 'pos1', '<='), ('end', 'pos1', '>='), use_numba=True)

105 ms ± 5.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Compare memory usage:

In [33]:
%memit d1.merge(d2, how = 'cross').query("start <= pos1 <= end")

peak memory: 2589.00 MiB, increment: 1868.00 MiB


In [34]:
%memit d1.conditional_join(d2, ('start', 'pos1', '<='), ('end', 'pos1', '>='), use_numba=False)

peak memory: 605.42 MiB, increment: -5.44 MiB


In [35]:
%memit d1.conditional_join(d2, ('start', 'pos1', '<='), ('end', 'pos1', '>='), use_numba=True)

peak memory: 602.98 MiB, increment: -2.44 MiB


Let's run another performance test - the code below is a self join to find overlapping events in a 30,000 row DataFrame, and adapted from [DuckDB](https://github.com/duckdb/duckdb/blob/master/benchmark/micro/join/iejoin_events.benchmark):

In [36]:
url = 'https://github.com/samukweku/data-wrangling-blog/raw/master/_notebooks/Data_files/results.csv'
events = pd.read_csv(url, parse_dates=['start', 'end']).iloc[:, 1:]
events.head()

,id,name,audience,start,sponsor,end
0,1,Event 1,1178,2022-11-19 10:00:00,Sponsor 2,2022-11-19 10:15:00
1,2,Event 2,1446,2015-09-27 15:00:00,Sponsor 11,2015-09-27 15:11:00
2,3,Event 3,2261,2019-11-12 18:00:00,Sponsor 10,2019-11-12 18:53:00
3,4,Event 4,1471,2019-12-24 22:00:00,Sponsor 6,2019-12-24 22:11:00
4,5,Event 5,2605,2028-06-20 12:00:00,Sponsor 8,2028-06-20 12:31:00


In [37]:
(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    ('id', 'id', '!='),
    use_numba = False,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)

left                                          right  \
         id               start                 end     id   
0        10 1993-11-27 12:00:00 1993-11-27 12:37:00   2345   
1        15 1993-04-04 16:00:00 1993-04-04 18:00:00  11178   
2        17 2030-10-25 07:00:00 2030-10-25 07:27:00  19605   
3        26 2005-10-04 17:00:00 2005-10-04 17:18:00   8218   
4        35 2024-05-02 15:00:00 2024-05-02 15:35:00   6916   
...     ...                 ...                 ...    ...   
3697  29966 2000-08-26 11:00:00 2000-08-26 13:00:00  29375   
3698  29971 2018-05-18 04:00:00 2018-05-18 04:18:00  24173   
3699  29978 1992-06-07 22:00:00 1992-06-07 22:23:00    981   
3700  29984 2025-06-05 03:00:00 2025-06-05 03:17:00  19051   
3701  29995 2016-09-04 14:00:00 2016-09-04 14:32:00  12296   

                                              
                   start                 end  
0    1993-11-27 10:00:00 1993-11-27 12:00:00  
1    1993-04-04 17:00:00 1993-04-04 17:22:00  
2    2030-10-25 06:00:00 2030-10-25 08:00:00  
3    2005-10-04 17:00:00 2005-10-04 17:27:00  
4    2024-05-02 15:00:00 2024-05-02 15:36:00  
...                  ...                 ...  
3697 2000-08-26 13:00:00 2000-08-26 13:53:00  
3698 2018-05-18 04:00:00 2018-05-18 04:36:00  
3699 1992-06-07 22:00:00 1992-06-07 22:30:00  
3700 2025-06-05 01:00:00 2025-06-05 03:00:00  
3701 2016-09-04 14:00:00 2016-09-04 14:50:00  

[3702 rows x 6 columns]

The cartesian join takes a very long time - too long for us to test here. The focus for this test will just be on [conditional_join](https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.conditional_join.conditional_join) and performance when [numba](https://numba.readthedocs.io/en/stable/) is used.

In [38]:
%%timeit
(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    ('id', 'id', '!='),
    use_numba = False,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)

1.19 s ± 46.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [39]:
%%timeit
(events
.conditional_join(
    events,
    ('start', 'end', '<='),
    ('end', 'start', '>='),
    ('id', 'id', '!='),
    use_numba = True,
    df_columns = ['id', 'start', 'end'],
    right_columns = ['id', 'start', 'end'])
)

25 ms ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Let's look at performance when an equi join is present:

In [40]:
from string import ascii_lowercase
np.random.seed(1)
n = 20_000; k = 100_000

idx1 = np.random.randint(0, high = 100, size = n)
idx2 = np.random.randint(0, high = 100, size = n)

d1 = dict(x = np.random.choice(list(ascii_lowercase[:5]), size=n),
          start = np.minimum(idx1, idx2),
          end = np.maximum(idx1, idx2))

d2 = dict(x = np.random.choice(list(ascii_lowercase[:15]), size=k),
          pos1 = np.random.randint(low=60, high = 151, size=k))

d1 = pd.DataFrame(d1)
d2 = pd.DataFrame(d2)

In [41]:
out = (d1
       .merge(d2, on = 'x')
       .query('start <= pos1 <= end')
       )
out.shape

(17792800, 4)

In [42]:
out = (d1
       .conditional_join(
            d2, 
            ('start', 'pos1', '<='), 
            ('end', 'pos1', '>='), 
            ('x', 'x', '=='), 
            use_numba=False)
    )
out.shape

(17792800, 5)

About 18 million rows returned; let's see how fast each function runs:

In [43]:
%%timeit
(d1
.merge(d2, on = 'x')
.query('start <= pos1 <= end')
)

25.1 s ± 2.81 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%%timeit
(d1
.conditional_join(
    d2, 
    ('start', 'pos1', '<='), 
    ('end', 'pos1', '>='), 
    ('x', 'x', '=='), 
    use_numba=False)
)

9.55 s ± 763 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
%memit d1.merge(d2, on = 'x').query('start <= pos1 <= end')

peak memory: 6334.26 MiB, increment: 5885.17 MiB


In [46]:
%memit d1.conditional_join(d2, ('start', 'pos1', '<='), ('end', 'pos1', '>='), ('x', 'x', '=='), use_numba=False)

peak memory: 6392.00 MiB, increment: 6150.73 MiB


[conditional_join](https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.conditional_join.conditional_join) is faster; this might not be the case all the time. 

## **Summary**

This blog post shows how to efficiently join on non-equi conditions, using [conditional_join](https://pyjanitor-devs.github.io/pyjanitor/api/functions/#janitor.functions.conditional_join.conditional_join). If you are comfortable with SQL, you could get more performance with [DuckDB](https://duckdb.org), which supports non-equi joins as well, and has a [performant implementation](https://duckdb.org/2022/05/27/iejoin.html) for range joins.